# [모듈 6.1] 세이지메이커 모델 스텝 개발

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [25]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 
%store

Stored variables and their in-db values:
base_preproc_input_dir                 -> 'opt/ml/processing/input'
dataset_path                           -> 'opt/ml/processing/input/dataset.csv'
default_bucket                         -> 'sagemaker-ap-northeast-2-057716757052'
image_uri                              -> '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com
input_data_uri                         -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
preprocessing_code_dir                 -> 'fraud/preprocessing.py'
processing_instance_count              -> ParameterInteger(name='ProcessingInstanceCount', p
project_prefix                         -> 'fraud2scratch'
s3_dataset_path                        -> 's3://sagemaker-ap-northeast-2-057716757052/fraud2
sagemaker_model                        -> 'pipelines-z2fiqp2pcp83-fraudscratchmodel-jvft2mid
test_preproc__dir_artifact             -> 's3://sagemaker-ap-northeast-2-057716757052/sklear
train_model_artifact                   -> 's3:

## 1. 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [26]:
# from sagemaker.workflow.parameters import (
#     ParameterInteger,
#     ParameterString,
# )

# processing_instance_count = ParameterInteger(
#     name="ProcessingInstanceCount",
#     default_value=1
# )
# processing_instance_type = ParameterString(
#     name="ProcessingInstanceType",
#     default_value="ml.m5.xlarge"
# )

### 추론 실행에 사용할 모델 정의 

모델을 사용하는 배치변환작업을 실행하기 위해 SageMaker Model을 생성합니다.

구체적으로, `TrainingStep` 인스턴스인 `step_train`의 속성 중 `S3ModelArtifacts` 정보를 활용하게 됩니다. `TrainingStep`의 `properties` 속성은 [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) API의 응답결과와 동일합니다.


In [27]:
from sagemaker.model import Model

# sklearn_image_uri = '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

model = Model(
    image_uri=image_uri,
#     image_uri=sklearn_image_uri,    
    model_data= train_model_artifact,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [28]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    # accelerator_type="ml.eia1.medium",
)
step_create_model = CreateModelStep(
    name="FraudScratchModel",
    model=model,
    inputs=inputs,
)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [29]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
#         processing_instance_type, 
#         processing_instance_count,
    ],
    steps=[step_create_model],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [30]:
import json


definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [31]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/fraud2scratch',
 'ResponseMetadata': {'RequestId': '64bc66d4-2b3e-465d-8342-1669abce9ee2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '64bc66d4-2b3e-465d-8342-1669abce9ee2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Tue, 13 Apr 2021 11:59:55 GMT'},
  'RetryAttempts': 0}}

In [32]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [33]:
execution.describe()
execution.wait()

In [34]:
execution.list_steps()

[{'StepName': 'FraudScratchModel',
  'StartTime': datetime.datetime(2021, 4, 13, 11, 59, 56, 754000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 4, 13, 11, 59, 57, 990000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model/pipelines-n5qxc409wxod-fraudscratchmodel-jdpbccudlk'}}}]

In [35]:
response = execution.list_steps()
sagemaker_model = response[0]['Metadata']['Model']['Arn'].split("/")[-1]
print("sagemaker_model: \n", sagemaker_model)

%store sagemaker_model

sagemaker_model: 
 pipelines-n5qxc409wxod-fraudscratchmodel-jdpbccudlk
Stored 'sagemaker_model' (str)
